# Fabry perot cavity perutrbations debugging

In [9]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))
import pyperclip

def copy_parameters_func(local_parameters):
    l = []
    for key, value in local_parameters.items():
        # if key.startswith("t_") or key.startswith("p_") or key in ["t", "p"]:
            # print(f"{key} = 1j*{value/np.pi:.10e}")
        if isinstance(value, float):
            l.append(f"{key} = {value:.10e}")
        elif isinstance(value, str):
            l.append(f"{key} = '{value}'")
        else:
            l.append(f"{key} = {value}")
    final_expression = '\n'.join(l)
    pyperclip.copy(final_expression)

from cavity import *
from ipywidgets import Layout, interact, FloatSlider, Checkbox, Dropdown, Text
import ipywidgets as widgets


In [10]:
from cavity import *

params = [OpticalElementParams(name='None'                   ,surface_type='curved_mirror'                  , x=-4.999964994473332e-03  , y=0                       , z=0                       , theta=0                       , phi=-1e+00 * np.pi          , r_1=5e-03                   , r_2=np.nan                  , curvature_sign=CurvatureSigns.concave, T_c=np.nan                  , n_inside_or_after=1e+00                   , n_outside_or_before=1e+00                   , diameter=np.nan                  , material_properties=MaterialProperties(refractive_index=1.45e+00                , alpha_expansion=5.2e-07                 , beta_surface_absorption=1e-06                   , kappa_conductivity=1.38e+00                , dn_dT=1.2e-05                 , nu_poisson_ratio=1.6e-01                 , alpha_volume_absorption=1e-03                   , intensity_reflectivity=1e-04                   , intensity_transmittance=9.99899e-01             , temperature=np.nan                  )),
          OpticalElementParams(name='None'                   ,surface_type='curved_mirror'                  , x=4.999964994473332e-03   , y=0                       , z=0                       , theta=0                       , phi=0                       , r_1=5e-03                   , r_2=np.nan                  , curvature_sign=CurvatureSigns.concave, T_c=np.nan                  , n_inside_or_after=1e+00                   , n_outside_or_before=1e+00                   , diameter=np.nan                  , material_properties=MaterialProperties(refractive_index=1.45e+00                , alpha_expansion=5.2e-07                 , beta_surface_absorption=1e-06                   , kappa_conductivity=1.38e+00                , dn_dT=1.2e-05                 , nu_poisson_ratio=1.6e-01                 , alpha_volume_absorption=1e-03                   , intensity_reflectivity=1e-04                   , intensity_transmittance=9.99899e-01             , temperature=np.nan                  )),]



def f(add_unheated_cavity, copy_input_parameters, copy_cavity_parameters, print_perturbations, power_laser,
      camera_center, x_span, y_span,
      element_index_0, param_name_0, perturbation_value_special_log_0, perturbation_value_special_log_0_fine,
      element_index_1, param_name_1, perturbation_value_special_log_1, perturbation_value_special_log_1_fine, eval_box,
      ):

    if copy_input_parameters:
        copy_parameters_func(locals())
        
    perturbation_value_0 = widget_convenient_exponent(perturbation_value_special_log_0, base=10, scale=-10)
    perturbation_value_1 = widget_convenient_exponent(perturbation_value_special_log_1, base=10, scale=-10)

    perturbation_value_0_fine = widget_convenient_exponent(perturbation_value_special_log_0_fine, base=10, scale=-10)
    perturbation_value_1_fine = widget_convenient_exponent(perturbation_value_special_log_1_fine, base=10, scale=-10)

    perturbation_value_0 += perturbation_value_0_fine
    perturbation_value_1 += perturbation_value_1_fine
    
    cavity = Cavity.from_params(params=params, standing_wave=True,
                                lambda_0_laser=LAMBDA_0_LASER, power=power_laser, p_is_trivial=True, t_is_trivial=True, use_paraxial_ray_tracing=False, set_central_line=True, set_mode_parameters=True)
    perturbation_pointers = [
                             PerturbationPointer(element_index=element_index_0, parameter_name=param_name_0, perturbation_value=perturbation_value_0),
                             PerturbationPointer(element_index=element_index_1, parameter_name=param_name_1, perturbation_value=perturbation_value_1)
                            ]
    perturbed_cavity = perturb_cavity(cavity=cavity, perturbation_pointer=perturbation_pointers)
    if eval_box != '':
        try:
            exec(f"print({eval_box})")
        except (NameError, AttributeError) as e:
            print(f'invalid expression: {e}')
    fig, ax = plt.subplots(3, 1, figsize=(8, 12))
    perturbed_cavity.plot(ax=ax[0])
    perturbed_cavity.plot(ax=ax[1])
    spot_size_lines_original = cavity.generate_spot_size_lines(dim=2, plane='xy')
    for line in spot_size_lines_original:
                ax[0].plot(line[0, :], line[1, :], color='green', linestyle='--', alpha=0.8, linewidth=0.5, label="perturbed_mode")
                ax[1].plot(line[0, :], line[1, :], color='green', linestyle='--', alpha=0.8, linewidth=0.5, label="perturbed_mode")
    
    plot_2_cavity_perturbation_overlap(cavity=cavity, second_cavity=perturbed_cavity, real_or_abs = 'real', ax=ax[2])
    u = np.linalg.norm(perturbed_cavity.physical_surfaces[0].origin - perturbed_cavity.physical_surfaces[1].origin)
    NA_of_u = np.sqrt(2 * LAMBDA_0_LASER / np.pi) * (perturbed_cavity.arms[0].central_line.length * u) ** (-1/4)
    plt.suptitle(f"param_name_0={param_name_0}, {perturbation_value_0=:.3e}, param_name_1={param_name_1}, {perturbation_value_1=:.3e}\nNA (target)={perturbed_cavity.mode_parameters[0].NA[0]:.3e}, u (extracted)={u:.3e}, NA (extracted from u) = {NA_of_u:.3e}")
    ax[1].set_ylim(-10**y_span, 10**y_span)
    ax[1].set_xlim(-10**x_span, 10**x_span)
    ax[1].scatter([perturbed_cavity.physical_surfaces[0].origin[0], perturbed_cavity.physical_surfaces[1].origin[0]], [perturbed_cavity.physical_surfaces[0].origin[1], perturbed_cavity.physical_surfaces[1].origin[1]], s=10)
    fig.tight_layout()
    plt.show()
    
    print(NA_of_u)
    if print_perturbations:
        final_value_0 = getattr(cavity.params[element_index_0], param_name_0)
        final_value_1 = getattr(cavity.params[element_index_1], param_name_1)
        print(f"{final_value_0=:.3e}\n{final_value_1=:.3e}")
    if copy_cavity_parameters:
        pyperclip.copy(perturbed_cavity.formatted_textual_params)

    

small_boundary = 1e-6
medium_boundary = 1e-3
big_boundary = 1
dx = 1e-10
dt_max = 200
dtheta_max = 100

default_axis_span = 0.01
default_lambda_0_laser = 1064e-09
default_elev = 38.00
default_azim = 168.00
default_axis_span = 0.00
default_dim = 2
default_set_initial_surface = False
default_camera_center = -1


widgets.interact(f,
                 add_unheated_cavity=widgets.Checkbox(value=False, description='Add unheated cavity', disabled=False),
                 copy_input_parameters=widgets.Checkbox(value=False, description='Copy input parameters', disabled=False),
                 copy_cavity_parameters=widgets.Checkbox(value=False, description='Copy cavity parameters', disabled=False),
                 print_perturbations=widgets.Checkbox(value=True, description='Print perturbations', disabled=False),
                 power_laser=FloatSlider(value=50000, min=0, max=100000, step=1e2, description='power laser', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.3e', layout=Layout(width='500px')),
                 camera_center=widgets.Dropdown(options=[("Middle of cavity", -1), ('First', 0), ('Second', 1), ('Third', 2), ("Fourth", 3)], value=default_camera_center, description='Focus on mirror:',),
                 x_span=FloatSlider(value=-6, min=-6, max=-2.2, step=0.0001, description='x span', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.3e', layout=Layout(width='2000px')),
                 y_span=FloatSlider(value=-7, min=-7, max=-2.2, step=0.0001, description='y span', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.3e', layout=Layout(width='2000px')),
                 element_index_0=widgets.Dropdown(options=[('Left mirror', 0), ('Right mirror', 1)], value=0, description='First perturbation element:', style={'description_width': 'initial'}),
                 param_name_0=widgets.Dropdown(options=['x', 'y', 'theta', 'phi', 'r_1', 'r_2', 'n_inside_or_after', 'T_c', 'z'], value='x', description='First perturbation parameter:', style={'description_width': 'initial'}),
                 perturbation_value_special_log_0=FloatSlider(value=0, min=-10, max=10, step=1e-9, description='First perturbation value', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.3e', layout=Layout(width='2000px'), style={'description_width': 'initial'}),
                 perturbation_value_special_log_0_fine=FloatSlider(value=0, min=-10, max=10, step=1e-9, description='First perturbation value - fine', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.3e', layout=Layout(width='2000px'), style={'description_width': 'initial'}),
                 element_index_1=widgets.Dropdown(options=[('Left mirror', 0), ('Right mirror', 1)], value=0, description='Second perturbation element:', style={'description_width': 'initial'}),
                 param_name_1=widgets.Dropdown(options=['x', 'y', 'theta', 'phi', 'r_1', 'r_2', 'n_inside_or_after', 'T_c', 'z'], value='x', description='Second perturbation parameter:', style={'description_width': 'initial'}),
                 perturbation_value_special_log_1=FloatSlider(value=0, min=-10, max=10+1e-9, step=1e-9, description='Second perturbation value', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.3e', layout=Layout(width='2000px'), style={'description_width': 'initial'}),
                 perturbation_value_special_log_1_fine=FloatSlider(value=0, min=-10, max=10+1e-9, step=1e-9, description='Second perturbation value - fine', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.3e', layout=Layout(width='2000px'), style={'description_width': 'initial'}),
                 eval_box=Text(value='', placeholder='Type a Python expression to print (e.g., cavity.arms[0].length)', description='Evaluate:', layout=Layout(width='1500px'), style={'description_width': 'initial'})
                 );

interactive(children=(Checkbox(value=False, description='Add unheated cavity'), Checkbox(value=False, descript…